In [ ]:
#Importing the OptKnock and production envelope modules from cameo
from cameo.strain_design import OptKnock
from cameo import load_model
from cameo import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

In [ ]:
#Loading the MEV (or MEP) metabolic model
model = load_model('iJO1366_MEV.xml')
#model = load_model('iJO1366_MEP.xml')

#Knocking out the MEP pathway starting reaction so only the MEV pathway is used
#Don't need this bit if you're using the MEP model
model.reactions.get_by_id("DXPRIi").knock_out()

In [ ]:
#Setting gurobi as the solver for the model
model.solver = 'gurobi'

#Solving the FBA for wt (before knockout)
wt_solution = model.optimize()

#The fluxes for reaction of WT model
growth = wt_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

#The flux through the beta-carotene sink reaction
caro_production = wt_solution.fluxes["DM_CRTY"]

In [ ]:
#Set up plotlyplotter due to some problems of running the plot function and it not recognising plotly
plotter = PlotlyPlotter()

#Set up the production enevelope for the WT model
p = phenotypic_phase_plane(model, variables=['BIOMASS_Ec_iJO1366_core_53p95M'], objective='DM_CRTY')
p.plot(plotter, points=[(growth, caro_production)])

In [ ]:
#Creating the OptKnock model
optknock = OptKnock(model)

In [ ]:
#Running OptKnock, it can be done for multiple reactions
result = optknock.run(max_knockouts=1,
                      target="DM_CRTY",
                      biomass="BIOMASS_Ec_iJO1366_core_53p95M")
result

In [ ]:
#A comparison of production envelopes for knockout (MT) and WT models
result.plot(plotter, 0)

#As can be seen no knockout will increase the production while at the same time increasing growth rate
#and the only possible knockouts are for essential reactions, that jeapordize the growth of the organism.